<a href="https://colab.research.google.com/github/nadunchanna98/machine-learning/blob/main/SVM_with_Linear_%2C_Polynomial%2C_RBF_kernel_and_K_fold_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2019/E/094

##### SVM for classification.Using different kernels in SVM.
##### Introducing K-fold Cross validation

#### 1. You are requested to use HCV dataset directly from UCI data repository.

In [358]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score

In [359]:
# Step 2: Dataset Preparation
# Load the HCV dataset from UCI repository

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv'
data = pd.read_csv(url,na_values=['NA'])

In [360]:
data.shape

(615, 14)

In [361]:
data.head()

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [362]:
data.columns

Index(['Unnamed: 0', 'Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST',
       'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT'],
      dtype='object')

In [363]:
data.drop('Unnamed: 0', axis=1,inplace=True)

In [364]:
data.tail()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
610,3=Cirrhosis,62,f,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
611,3=Cirrhosis,64,f,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
612,3=Cirrhosis,64,f,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
613,3=Cirrhosis,46,f,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0
614,3=Cirrhosis,59,f,36.0,NaN,100.0,80.0,12.0,9.07,5.30,67.0,34.0,68.0


#### 2. Prepare your dataset for learning.

In [365]:
# check missing values
data.isna()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,False,False,False,False,False,False,False,False,False,False,False,False,False
611,False,False,False,False,False,False,False,False,False,False,False,False,False
612,False,False,False,False,False,False,False,False,False,False,False,False,False
613,False,False,False,False,True,False,False,False,False,False,False,False,False


In [366]:
#detect the columns contains missing values
data.isna().any()

Category    False
Age         False
Sex         False
ALB          True
ALP          True
ALT          True
AST         False
BIL         False
CHE         False
CHOL         True
CREA        False
GGT         False
PROT         True
dtype: bool

In [367]:
#check whether the missing of all values in the columns
data.isna().all()

Category    False
Age         False
Sex         False
ALB         False
ALP         False
ALT         False
AST         False
BIL         False
CHE         False
CHOL        False
CREA        False
GGT         False
PROT        False
dtype: bool

In [368]:
#check whether the missing of all values in the raws
data.isna().all(axis=1).head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [369]:
#presentage of mising values in a column
print(data.isna().sum()/len(data)*100)

Category    0.000000
Age         0.000000
Sex         0.000000
ALB         0.162602
ALP         2.926829
ALT         0.162602
AST         0.000000
BIL         0.000000
CHE         0.000000
CHOL        1.626016
CREA        0.000000
GGT         0.000000
PROT        0.162602
dtype: float64


In [370]:
# missing values filling using mean of each mean value
data2=data["ALB"].fillna(data["ALB"].mean())
data3=data["ALP"].fillna(data["ALP"].mean())
data4=data["ALT"].fillna(data["ALT"].mean())
data5=data["CHOL"].fillna(data["CHOL"].mean())
data6=data["PROT"].fillna(data["PROT"].mean())

In [371]:
data2.head()

0    38.5
1    38.5
2    46.9
3    43.2
4    39.2
Name: ALB, dtype: float64

In [372]:
data['ALB'] = data2
data['ALP'] = data3
data['ALT'] = data4
data['CHOL'] = data5
data['PROT'] = data6

In [373]:
print(data.isna().sum()/len(data)*100)

Category    0.0
Age         0.0
Sex         0.0
ALB         0.0
ALP         0.0
ALT         0.0
AST         0.0
BIL         0.0
CHE         0.0
CHOL        0.0
CREA        0.0
GGT         0.0
PROT        0.0
dtype: float64


##### NaN : means 0/0 -- Stands for Not a Number NA : is generally interpreted as a missing, does not exist NULL : is for empty object

In [374]:
#now no missing values. now replace categorical to numerical
# check data types
data_types = data.dtypes
print(data_types)

Category     object
Age           int64
Sex          object
ALB         float64
ALP         float64
ALT         float64
AST         float64
BIL         float64
CHE         float64
CHOL        float64
CREA        float64
GGT         float64
PROT        float64
dtype: object


In [375]:
# Select columns with categorical data types
categorical_columns = data.select_dtypes(include=['object', 'category']).columns
categorical_columns

Index(['Category', 'Sex'], dtype='object')

In [376]:
#In the same column, count the number of instances in each value:
no_of_instances_Category = data['Category'].value_counts()
no_of_instances_Sex = data['Sex'].value_counts()
no_of_instances_Category

0=Blood Donor             533
3=Cirrhosis                30
1=Hepatitis                24
2=Fibrosis                 21
0s=suspect Blood Donor      7
Name: Category, dtype: int64

In [377]:
no_of_instances_Sex

m    377
f    238
Name: Sex, dtype: int64

In [378]:
#Change all the categorical values into numeric values:
from sklearn.preprocessing import LabelEncoder

# Apply Label Encoding to each categorical column
label_encoder = LabelEncoder()
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column].astype(str))

data.head()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,0,32,1,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,0,32,1,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,0,32,1,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,0,32,1,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,0,32,1,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [379]:
data_types = data.dtypes
print(data_types)

Category      int64
Age           int64
Sex           int64
ALB         float64
ALP         float64
ALT         float64
AST         float64
BIL         float64
CHE         float64
CHOL        float64
CREA        float64
GGT         float64
PROT        float64
dtype: object


#### 3. First, Split your dataset into training set and test set.

In [380]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = data.drop('Category', axis=1)
y = data['Category']

In [381]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 3. Build SVM using following kernels and measure the performance of your model.

##### (a) Linear Kernel
##### (b) Polynomial Kernel
##### (c) RBF kernel


##### **In machine learning, a kernel refers to a method that allows us to apply linear classifiers to non-linear problems by mapping non-linear data into a higher-dimensional space without the need to visit or understand that higher-dimensional space. **

In [382]:
# (a) Linear Kernel
svm_linear = svm.SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
accuracy_linear = accuracy_score(y_test, y_pred_linear)

# (b) Polynomial Kernel
svm_poly = svm.SVC(kernel='poly')
svm_poly.fit(X_train, y_train)
y_pred_poly = svm_poly.predict(X_test)
accuracy_poly = accuracy_score(y_test, y_pred_poly)

# (c) RBF Kernel
svm_rbf = svm.SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

print("Accuracy with Linear Kernel:", accuracy_linear)
print("Accuracy with Polynomial Kernel:", accuracy_poly)
print("Accuracy with RBF Kernel:", accuracy_rbf)

Accuracy with Linear Kernel: 0.8455284552845529
Accuracy with Polynomial Kernel: 0.8130081300813008
Accuracy with RBF Kernel: 0.8048780487804879


#### 4. Now, do the same exercise using 10-fold and 5-fold cross validation and interpret your result in the proper way.


In [383]:
# Step 5: K-Fold Cross-Validation

# (a) 10-fold Cross-Validation
kf_10 = KFold(n_splits=10, shuffle=True, random_state=42)

svm_linear_scores = cross_val_score(svm_linear, X, y, cv=kf_10)
svm_poly_scores = cross_val_score(svm_poly, X, y, cv=kf_10)
svm_rbf_scores = cross_val_score(svm_rbf, X, y, cv=kf_10)

print("Average Accuracy with Linear Kernel (10-fold CV):", svm_linear_scores.mean())
print("Average Accuracy with Polynomial Kernel (10-fold CV):", svm_poly_scores.mean())
print("Average Accuracy with RBF Kernel (10-fold CV):", svm_rbf_scores.mean())

Average Accuracy with Linear Kernel (10-fold CV): 0.9009518773135907
Average Accuracy with Polynomial Kernel (10-fold CV): 0.9010576414595451
Average Accuracy with RBF Kernel (10-fold CV): 0.9026441036488629


In [384]:
# (b) 5-fold Cross-Validation
kf_5 = KFold(n_splits=5, shuffle=True, random_state=42)

svm_linear_scores_5fold = cross_val_score(svm_linear, X, y, cv=kf_5)
svm_poly_scores_5fold = cross_val_score(svm_poly, X, y, cv=kf_5)
svm_rbf_scores_5fold = cross_val_score(svm_rbf, X, y, cv=kf_5)

print("Average Accuracy with Linear Kernel (5-fold CV):", svm_linear_scores_5fold.mean())
print("Average Accuracy with Polynomial Kernel (5-fold CV):", svm_poly_scores_5fold.mean())
print("Average Accuracy with RBF Kernel (5-fold CV):", svm_rbf_scores.mean())

Average Accuracy with Linear Kernel (5-fold CV): 0.9056910569105691
Average Accuracy with Polynomial Kernel (5-fold CV): 0.9056910569105691
Average Accuracy with RBF Kernel (5-fold CV): 0.9026441036488629
